In [1]:
import os
import glob
import pandas as pd
import time
import matplotlib.pyplot as plt

In [2]:
plt.rcParams['figure.figsize'] = [35, 15]

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
data_path = "/home/srivasta/shared/us_presidential_election_2020/SimpleExtractionFeatures/vp_debate_version2"
os.chdir(data_path)

In [7]:
#specify datatypes so that datatype guessing isnt done (looks like he has mentioned default returned value as 'none' string everywhere)
dtype={'UID':int,'Hashtags':object,'hashtag_content': object,'new_content':object,'Tweet Text':object}
df = pd.read_csv('SED/hashtag_users.csv',dtype=dtype,lineterminator='\n')

In [8]:
df.head()

,Hashtags,UID,Tweet Text,new_content,hashtag_content
0,[],629751550,"RT @OliviaTroye (2 of 3) This is the same group of people mocking @KamalaHarris for taking extra precautions for the VP debate. ""If Sen. Harris wants to use a fortress around herself, have at it,"" said @VPComDir Katie Miller, who had Covid in May...",[],"('empty', 0, [])"
1,[],1307358119782047745,@tinabelle77 @realDonaldTrump I’m sure he’s called his colleagues in person and wished them well I’m sure he’s maki… https://t.co/cd2N5B4PNF,[],"('empty', 0, [])"
2,[],1217552218116775941,RT @ThePubliusUSA Is Mike Pence trying to finagle his way out of the debate because he is terrified of being made minced-meat on live television by @KamalaHarris?,[],"('empty', 0, [])"
3,[],1207400068510371840,"RT @StevenBeschloss Mike Pence, head of the coronavirus task force, refuses protective plexiglass for the VP debate. The infected Donald Trump refuses to clarify when he first tested positive. His regime refuses to allow the CDC to perform contact tracing. This isn’t politics. This is deadly sick.",[],"('empty', 0, [])"
4,[],36185294,"RT @GeorgeTakei On a very serious note, Stephen Miller tested negative for five days before finally testing positive. This disease incubates. We have to keep Kamala Harris safe from Mike “Disease Gang” Pence.",[],"('empty', 0, [])"


In [13]:
with open("SED/democrat_tags.txt") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
dem_list = [x.strip() for x in content] 

with open("SED/republican_tags") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
rep_list = [x.strip() for x in content] 

with open("SED/independent_tags.txt") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
ind_list = [x.strip() for x in content] 

with open("SED/covid_tags.txt") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
cov_list = [x.strip() for x in content] 


In [15]:
def set_flags(row):
    flag_list =[0,0,0,0,0]
    uid = row['UID']
    tup = eval(row['hashtag_content'])
    if tup[1]==0:
        return "no hashtag"
    else:
        hashtag_list = tup[2]
        for hashtag in hashtag_list:
            if hashtag in dem_list:
                flag_list[0]=1
            elif hashtag in rep_list:
                flag_list[1]=1
            elif hashtag in ind_list:
                flag_list[2]=1
            elif hashtag in cov_list:
                flag_list[3]=1
            else:
                #miscellaneous cases not caught by my categorization
                flag_list[4]=1
        return flag_list
                
                
    

In [16]:
df['flags']= df.apply(lambda x: set_flags(x),axis=1)

In [18]:
df.shape

(7641457, 6)

In [17]:
df['flags'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


no hashtag         6302242
[0, 0, 0, 0, 1]    1237715
[1, 0, 0, 0, 1]      36856
[0, 1, 0, 0, 1]      14625
[1, 0, 0, 0, 0]      12307
[0, 0, 1, 0, 1]      11002
[0, 1, 0, 0, 0]      10122
[0, 0, 0, 1, 1]       6313
[0, 0, 0, 1, 0]       2962
[0, 0, 1, 0, 0]       1685
[1, 0, 1, 0, 1]       1397
[1, 1, 0, 0, 1]       1253
[0, 1, 1, 0, 1]        811
[0, 1, 0, 1, 1]        618
[1, 1, 0, 0, 0]        497
[1, 1, 0, 1, 1]        348
[0, 1, 0, 1, 0]        166
[1, 1, 1, 0, 1]        139
[1, 0, 0, 1, 1]        119
[0, 1, 1, 0, 0]        113
[1, 0, 1, 0, 0]         60
[0, 1, 1, 1, 1]         30
[0, 0, 1, 1, 1]         28
[1, 0, 0, 1, 0]         23
[1, 1, 1, 0, 0]         10
[0, 1, 1, 1, 0]          5
[1, 0, 1, 1, 1]          5
[1, 1, 0, 1, 0]          3
[0, 0, 1, 1, 0]          3
Name: flags, dtype: int64

In [29]:
import json
def get_party_affiliation(row):
    if row == "no hashtag":
        return "no hashtag"
    else:
        #print(row)
        #flag_list = json.loads(row)
        flag_list = row
        if flag_list[0]==1 and flag_list[1]==1:
            return "both"
        elif flag_list[0]==1 and flag_list[1]==0:
            return "democrat"
        elif flag_list[0]==0 and flag_list[1]==1:
            return "republican"
        else:
            return "none"
        

In [30]:
df['party_affiliation']= df.apply(lambda x: get_party_affiliation(x['flags']),axis=1)

In [31]:
df['party_affiliation'].value_counts()

no hashtag    6302242
none          1259708
democrat        50767
republican      26490
both             2250
Name: party_affiliation, dtype: int64